 # Predicting heart disease using machine learning
 
 This jupyter notebook uses python machine learning and data science libraries in order to build a machine learning model that can predict if someone has a heart disease or not based on medical attributes
 
 Approach: 
 1. Problem definition
 2. Data
 3. Evaluation
 4. Features
 5. Modelling 
 6. Experimentation

# 1. Problem definition

We are doing binary classification, that means we are prediction whether or not someone has a heart disease. Can it be made using a classification machine learning model?

# 2. Data

Data comes from Kaggle, it is a version of the Cleaveland UCI Machine Learning Repository. 
https://www.kaggle.com/rotnif/heart-disease-uci

# 3. Evaluation
> We are interested in building a high accuracy project given the problem in which
95% or more accuracy will be the target.

# 4. Features

***Data dictionary***
* age in years
* sex ( 1 = male; 0 = female)
* cp chest pain type
* trestbpsrtesting blood pressure (in mmHg on admission to the hospital)
* cholserum cholestoral in mg/dl
* fbs ( fasting blood sugar > 120 mg/dl) ( 1 = true, 0 = false)
* restecgresting electrocardiographic results
* thalachmaximum heart rate achieved
* exangexercise induced angina (1 = yes, 0 = no)
* oldpeakST deppresion induced by excercise relative to rest
* slopethe slope of the peak excercise ST segment
* canumber of major vessels (0-3) colored by flouroscopy
* thal3 = normal; 6 = fixed defect; 7 = reversable defect
* target 1 or 0